In [1]:
import os
import sys
import numpy as np
import neuroglancer

HOME = os.path.expanduser("~")
sys.path.append(os.path.join(HOME, 'programming/pipeline_utility'))

from utilities.utilities_cvat_neuroglancer import get_structure_number, NumpyToNeuroglancer, get_segment_properties


Connecting dklab@192.168.1.12:3306


In [2]:
surround = False
ATLAS_V7_DIR = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/score_volumes'
files = os.listdir(ATLAS_V7_DIR)
volume_files = sorted([f for f in files if f.endswith('.npy') and surround == ('surround' in f) and 'test' not in f])
origin_files = sorted([f for f in files if f.endswith('.txt') and surround == ('surround' in f) and 'test' not in f])
    
structure_volume_origin = {}
for volume_filename, origin_filename in zip(volume_files, origin_files):
    prefix = os.path.splitext(volume_filename)[0]
    structure = prefix.replace('atlasV7_10.0um_scoreVolume_', '').replace('_surround_200um', '')
    if structure not in origin_filename:
        print(structure, origin_filename)
        break

    color = get_structure_number(structure.replace('_L', '').replace('_R', ''))

    volume = np.load(os.path.join(ATLAS_V7_DIR, volume_filename))
    origin = np.loadtxt(os.path.join(ATLAS_V7_DIR, origin_filename))
    
    volume = np.rot90(volume, axes=(0,1))
    volume = np.flip(volume, axis=0)
    volume[volume > 0.8] = color
    volume = volume.astype(np.uint8)
    
    structure_volume_origin[structure] = (volume, origin)
print(structure_volume_origin.keys())

dict_keys([])


In [3]:
x_length = 1000
y_length = 1000
z_length = 300
atlasV7_volume = np.zeros((x_length, y_length, z_length), dtype=np.uint32)

for structure, (volume, origin) in structure_volume_origin.items():
    x, y, z = origin
    x_start = int(x) + x_length // 2
    y_start = int(y) + y_length // 2
    z_start = int(z) // 2 + z_length // 2
    x_end = x_start + volume.shape[0]
    y_end = y_start + volume.shape[1]
    z_end = z_start + (volume.shape[2] + 1) // 2

    z_indices = [z for z in range(volume.shape[2]) if z % 2 == 0]
    volume = volume[:, :, z_indices]
    atlasV7_volume[x_start:x_end, y_start:y_end, z_start:z_end] += volume

In [4]:
OUTPUT_DIR = os.path.join(HOME, 'atlas')

ng = NumpyToNeuroglancer(atlasV7_volume, [10000, 10000, 20000])
ng.preview()
ng.init_precomputed(OUTPUT_DIR)
ng.add_segment_properties(get_segment_properties())
ng.add_downsampled_volumes()
ng.add_segmentation_mesh()

A new layer named segmentation_[10000, 10000, 20000] is added to:
http://127.0.0.1:46249/v/0b89915ec10be548229110dc218ff8d23ec14635/


Downloading:  47%|████▋     | 121/256 [00:00<00:00, 885.61it/s]

Volume Bounds:  Bbox([0, 0, 0],[1000, 1000, 300], dtype=int32)
Selected ROI:   Bbox([0, 0, 0],[1000, 1000, 300], dtype=int32)



Downloading: 100%|██████████| 256/256 [00:00<00:00, 885.61it/s]

Uploading:   0%|          | 0/64 [00:00<?, ?it/s]

Uploading:  41%|████      | 26/64 [00:00<00:00, 244.24it/s]

Uploading:  41%|████      | 26/64 [00:00<00:00, 244.24it/s]

Uploading:  41%|████      | 26/64 [00:00<00:00, 244.24it/s]

Uploading:  97%|█████████▋| 62/64 [00:00<00:00, 282.51it/s]

Uploading:  98%|█████████▊| 63/64 [00:00<00:00, 289.71it/s]

Uploading:  98%|█████████▊| 63/64 [00:00<00:00, 289.71it/s]

Uploading:  98%|█████████▊| 63/64 [00:00<00:00, 289.71it/s]

Uploading:  98%|█████████▊| 63/64 [00:00<00:00, 289.71it/s]

Uploading:  98%|█████████▊| 63/64 [00:00<00:00, 289.71it/s]


Uploading:   0%|          | 0/16 [00:00<?, ?it/s]


Uploading: 100%|██████████| 16/16 [00:00<00:00, 101.97it/s]


Uploading: 100%|██████████| 16/16 [00:00<00:00, 101.97it/s]


Uploading: 100%|██████████| 16/16 [00:00<00:00, 101.97it/s]


Uploading: 100%|██████████| 16/16 [00:00<00:00, 101.97it/s]


Uploading: 100%|██████████| 16/16

Downloading: 100%|██████████| 256/256 [00:00<00:00, 37.73it/s]







Downloading: 100%|██████████| 256/256 [00:00<00:00, 37.73it/s]







Downloading: 100%|██████████| 256/256 [00:00<00:00, 37.73it/s]







Downloading: 100%|██████████| 256/256 [00:00<00:00, 37.73it/s]







Downloading: 100%|██████████| 256/256 [00:00<00:00, 37.73it/s]







Downloading: 100%|██████████| 256/256 [00:00<00:00, 37.73it/s]







Downloading: 100%|██████████| 256/256 [00:00<00:00, 37.73it/s]







Downloading: 100%|██████████| 256/256 [00:00<00:00, 37.73it/s]







Downloading: 100%|██████████| 256/256 [00:00<00:00, 37.73it/s]







Downloading: 100%|██████████| 256/256 [00:00<00:00, 37.73it/s]







Downloading: 100%|██████████| 256/256 [00:00<00:00, 37.73it/s]







Downloading: 100%|██████████| 256/256 [00:00<00:00, 37.73it/s]







Downloading: 100%|██████████| 256/256 [00:00<00:00, 37.73it/s]







Downloading: 100%|██████████| 256/256 [00:00<00:00, 37.73it/s]







Uploading:  97%|████

Uploading: 100%|██████████| 64/64 [00:00<00:00, 322.76it/s]

Tasks:   0%|          | 0/9 [00:00<?, ?it/s]

Uploading: 100%|██████████| 4/4 [00:00<00:00, 20.09it/s][A


Downloading:  31%|███       | 98/320 [00:00<00:00, 858.20it/s]

Downloading:  32%|███▎      | 104/320 [00:00<00:00, 881.09it/s]

Downloading:  33%|███▎      | 106/320 [00:00<00:00, 881.09it/s]

Downloading:  35%|███▌      | 112/320 [00:00<00:00, 881.09it/s]

Downloading:  36%|███▌      | 115/320 [00:00<00:00, 881.09it/s]

Downloading:  41%|████▏     | 132/320 [00:00<00:00, 881.09it/s]

Downloading:  48%|████▊     | 154/320 [00:00<00:00, 881.09it/s]

Downloading:  52%|█████▏    | 167/320 [00:00<00:00, 881.09it/s]

Downloading:  59%|█████▉    | 190/320 [00:00<00:00, 881.09it/s]

Downloading:  70%|██████▉   | 223/320 [00:00<00:00, 881.09it/s]

Downloading:  76%|███████▌  | 242/320 [00:00<00:00, 881.09it/s]

Downloading:  80%|████████  | 256/320 [00:00<00:00, 881.09it/s]

Downloading:  88%|████████▊ | 283/320 [00:00<00:00, 8

Downloading:  90%|█████████ | 72/80 [00:00<00:00, 539.71it/s]








Downloading:  90%|█████████ | 72/80 [00:00<00:00, 539.71it/s]








Downloading:  90%|█████████ | 72/80 [00:00<00:00, 539.71it/s]








Downloading:  90%|█████████ | 72/80 [00:00<00:00, 539.71it/s]








Downloading:  90%|█████████ | 72/80 [00:00<00:00, 539.71it/s]








Downloading: 100%|██████████| 320/320 [00:00<00:00, 417.13it/s] 









Tasks:  89%|████████▉ | 8/9 [00:02<00:00,  3.05it/s]4.85it/s]

Tasks: 100%|██████████| 999/999 [00:23<00:00, 41.68it/s]


In [5]:
#with open('atlasV7.npy', 'wb') as file:
#    np.save(file, atlasV7_volume)